In [18]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};

<IPython.core.display.Javascript object>

##Activation functions
###**Sigmoid** and **tanh**

A very undesirable property of the sigmoid neuron is that when the neuron's activation **saturates** at either tail of 0 or 1, the gradient at these regions is almost zero. Recall that during backpropagation, this (local) gradient will be multiplied to the gradient of this gate's output for the whole objective. Therefore, if the local gradient is very small, it will effectively "kill" the gradient and almost no signal will flow through the neuron to its weights and recursively to its data. Additionally, one must pay extra caution when initializing the weights of sigmoid neurons to prevent saturation.

<img  src="images/CNN/W3SI91D3D66UX13DQ2DSWWSBGXUCGQ3W.png" width="75%"/>


###**ReLU**
The Rectified Linear Unit has become very popular in the last few years. It computes the function $f(x) = max(0, x)$

<img  src="images/CNN/ICEOPGALI36SCPP7L0UEULQBFIEJ44I1.png" width="75%"/>
+ It was found to greatly **accelerate** the convergence of stochastic gradient descent compared to the sigmoid/tanh functions.
+ Compared to tanh/sigmoid neurons that involve expensive operations (exponentials, etc.), the ReLU can be implemented by **simply** thresholding a matrix of activations at zero.
+ Unfortunately, ReLU units can be fragile during training and can **"die"**. For example, a large gradient flowing through a ReLU neuron could cause the weights to update in such a way that the neuron will never activate on any datapoint again. If this happens, then the gradient flowing through the unit will forever be zero from that point on. That is, the ReLU units can irreversibly die during training since they can get knocked off the data manifold. For example, you may find that as much as 40% of your network can be "dead" (i.e. neurons that never activativate across the entire training dataset) if the learning rate is set too high. With a proper setting of the learning rate this is less frequently an issue.

###**Leaky ReLU** and **Maxout**
+ **Leaky ReLU**： attempt to fix the "dying ReLU" problem. That is, the function computes $f(x) = 1(x \lt 0)(\alpha x) + 1(x \ge 0)(x)$ where $\alpha$ is a small constant.
+ **Maxout**: The Maxout neuron computes the function $max(w^T_1x + b_1, w^T_2 + b_2)$

##Weight Initialization

+ All zero initialization: False
+ Small random numbers: 
```python
W = 0.001* np.random.randn(D,H)
```
where **randn** samples from a zero mean, unit standard deviation gaussian. 
+ Calibrating the variances with 1/sqrt(n).
Consider the inner product \\(s = \sum\_i^n w\_i x\_i\\) between the weights \\(w\\) and input \\(x\\), which gives the raw activation of a neuron before the non-linearity. We can examine the variance of \\(s\\):

$$
\begin{align}
\text{Var}(s) &= \text{Var}(\sum_i^n w_ix_i) \\\\
&= \sum_i^n \text{Var}(w_ix_i) \\\\
&= \sum_i^n [E(w_i)]^2\text{Var}(x_i) + E[(x_i)]^2\text{Var}(w_i) + \text{Var}(x_i)\text{Var}(w_i) \\\\
&= \sum_i^n \text{Var}(x_i)\text{Var}(w_i) \\\\
&= \left( n \text{Var}(w) \right) \text{Var}(x)
\end{align}
$$

And since \\(\text{Var}(aX) = a^2\text{Var}(X)\\) for a random variable \\(X\\) and a scalar \\(a\\), this implies that we should draw from unit gaussian and then scale it by \\(a = \sqrt{1/n}\\), to make its variance \\(1/n\\). This gives the initialization 

```python
w = np.random.randn(n) / sqrt(n)
```

##Regularization

+ L2 and L1: pass
+ Max norm constraints

Enforcing the constraint by clamping the weight vector \\(\vec{w}\\) of every neuron to satisfy \\(\Vert \vec{w} \Vert\_2 < c\\). Typical values of \\(c\\) are on orders of 3 or 4.

+ Dropout 

While training, dropout is implemented by only keeping a neuron active with some probability $p$ (a hyperparameter), or setting it to zero otherwise.

<img  src="images/CNN/U8DDDQ6NYQ8XN2MWYRRDG9MPU75UILOD.png" width="70%"/>

```python
p = 0.5 # probability of keeping a unit active. higher = less dropout

def train_step(X):
  """ X contains the data """
  
  # forward pass for example 3-layer neural network
  H1 = np.maximum(0, np.dot(W1, X) + b1)
  U1 = np.random.rand(*H1.shape) < p # first dropout mask
  H1 *= U1 # drop!
  H2 = np.maximum(0, np.dot(W2, H1) + b2)
  U2 = np.random.rand(*H2.shape) < p # second dropout mask
  H2 *= U2 # drop!
  out = np.dot(W3, H2) + b3
  
  # backward pass: compute gradients... (not shown)
  # perform parameter update... (not shown)
  
def predict(X):
  # ensembled forward pass
  H1 = np.maximum(0, np.dot(W1, X) + b1) * p # NOTE: scale the activations
  H2 = np.maximum(0, np.dot(W2, H1) + b2) * p # NOTE: scale the activations
  out = np.dot(W3, H2) + b3
```

One particular form of regularization was found to be especially useful for dropout—constraining the norm of the incoming weight vector at each hidden unit to be upper bounded by a fixed constant $c$. In other words, if $w$ represents the vector of weights incident on any hidden unit, the neural network was optimized under the constraint $||w||_2 ≤ c$. This constraint was imposed during optimization by projecting w onto the surface of a ball of radius $c$, whenever $w$ went out of it.

<img  src="images/CNN/QKU21AWTPKTI7UC6HRYPU2J6JLEWOSQ5.png" width="70%"/>

##Architecture 

###Convolutional Layer

####Local Connectivity
When dealing with high-dimensional inputs such as images, as we saw above it is impractical to connect neurons to all neurons in the previous volume. Instead, we will connect each neuron to only a local region of the input volume. The spatial extent of this connectivity is a hyperparameter called the **receptive field** of the neuron.

Example 1. For example, suppose that the input volume has size [32x32x3], (e.g. an RGB CIFAR-10 image). If the receptive field is of size 5x5, then each neuron in the Conv Layer will have weights to a [5x5x3] region in the input volume, for a total of 5x5x3 = 75 weights. Notice that the the extent of the connectivity along the depth axis must be 3, since this is the depth of the input volume.

Example 2. Suppose an input volume had size [16x16x20]. Then using an example receptive field size of 3x3, every neuron in the Conv Layer would now have a total of 3x3x20 = 180 connections to the input volume. Notice that, again, the connectivity is local in space (e.g. 3x3), but full along the input depth (20).

<img  src="images/CNN/JPT1KUT63I16DCGICE9B7OW8PQE1R3OM.png" width="50%"/>

An example input volume in red (e.g. a 32x32x3 CIFAR-10 image), and an example volume of neurons in the first Convolutional layer. Each neuron in the convolutional layer is connected only to a local region in the input volume spatially, but to the full depth (i.e. all color channels). Note, there are multiple neurons (5 in this example) along the depth, all looking at the same region in the input.

####Spatial arrangement
the **depth**, **stride** and **zero-padding**.

####Filter 
All neurons in a single depth slice are using the same weight vector, so the forward pass of the CONV layer can in each depth slice be computed as a convolution of the neuron's weights with the input volume (Hence the name: **Convolutional Layer**). Therefore, it is common to refer to the sets of weights as a **filter** (or a kernel), which is convolved with the input. The result of this convolution is an activation map , and the set of activation maps for each different filter are stacked together along the depth dimension to produce the output volume .

<img  src="images/CNN/VGB5F9DA7FBGO6QHB1848SB7PY5RWTKP.png" width="70%"/>


In [1]:
from IPython.display import HTML
HTML('<iframe src="html/CNN/conv_demo/index.html" width=100% height=700px></iframe>')

###Pooling Layer
In addition to max pooling, the pooling units can also perform other functions, such as average pooling or even L2-norm pooling. Average pooling was often used historically but has recently fallen out of favor compared to the max pooling operation, which has been shown to work better in practice.

<img  src="images/CNN/Y7J29T7PD8UHACXXWGAY5QX14HN7LVBB.png" width="70%"/>

###Fully-connected layer

Neurons in a fully connected layer have full connections to all activations in the previous layer, as seen in regular Neural Networks. Their activations can hence be computed with a matrix multiplication followed by a bias offset. 

###ConvNet Architectures
The most common form of a ConvNet architecture stacks a few CONV-RELU layers, follows them with POOL layers, and repeats this pattern until the image has been merged spatially to a small size. At some point, it is common to transition to fully-connected layers. The last fully-connected layer holds the output, such as the class scores. In other words, the most common ConvNet architecture follows the pattern:

```
INPUT -> [[CONV -> RELU]*N -> POOL?]*M -> [FC -> RELU]*K -> FC
```

Example:VGGNet

```
INPUT: [224x224x3]        memory:  224*224*3=150K   weights: 0
CONV3-64: [224x224x64]  memory:  224*224*64=3.2M   weights: (3*3*3)*64 = 1,728
CONV3-64: [224x224x64]  memory:  224*224*64=3.2M   weights: (3*3*64)*64 = 36,864
POOL2: [112x112x64]  memory:  112*112*64=800K   weights: 0
CONV3-128: [112x112x128]  memory:  112*112*128=1.6M   weights: (3*3*64)*128 = 73,728
CONV3-128: [112x112x128]  memory:  112*112*128=1.6M   weights: (3*3*128)*128 = 147,456
POOL2: [56x56x128]  memory:  56*56*128=400K   weights: 0
CONV3-256: [56x56x256]  memory:  56*56*256=800K   weights: (3*3*128)*256 = 294,912
CONV3-256: [56x56x256]  memory:  56*56*256=800K   weights: (3*3*256)*256 = 589,824
CONV3-256: [56x56x256]  memory:  56*56*256=800K   weights: (3*3*256)*256 = 589,824
POOL2: [28x28x256]  memory:  28*28*256=200K   weights: 0
CONV3-512: [28x28x512]  memory:  28*28*512=400K   weights: (3*3*256)*512 = 1,179,648
CONV3-512: [28x28x512]  memory:  28*28*512=400K   weights: (3*3*512)*512 = 2,359,296
CONV3-512: [28x28x512]  memory:  28*28*512=400K   weights: (3*3*512)*512 = 2,359,296
POOL2: [14x14x512]  memory:  14*14*512=100K   weights: 0
CONV3-512: [14x14x512]  memory:  14*14*512=100K   weights: (3*3*512)*512 = 2,359,296
CONV3-512: [14x14x512]  memory:  14*14*512=100K   weights: (3*3*512)*512 = 2,359,296
CONV3-512: [14x14x512]  memory:  14*14*512=100K   weights: (3*3*512)*512 = 2,359,296
POOL2: [7x7x512]  memory:  7*7*512=25K  weights: 0
FC: [1x1x4096]  memory:  4096  weights: 7*7*512*4096 = 102,760,448
FC: [1x1x4096]  memory:  4096  weights: 4096*4096 = 16,777,216
FC: [1x1x1000]  memory:  1000 weights: 4096*1000 = 4,096,000

TOTAL memory: 24M * 4 bytes ~= 93MB / image (only forward! ~*2 for bwd)
TOTAL params: 138M parameters
```

AlexNet
<img  src="images/CNN/VCBTAM7V1WO31L94N3LFV4FIBPIYNIAB.png" width="70%"/>


##Tricks

+ Gradient Checks
$$
\frac{df(x)}{dx} = \frac{f(x + h) - f(x - h)}{2h} \hspace{0.1in} \text{(use instead)}
$$

$$
\frac{\mid f'_a - f'_n \mid}{\mid f'_a \mid + \mid f'_n \mid}
$$

+ Overfit a tiny subset of data

+ Loss function
<img  src="images/CNN/LXA5VV6B41S41T2GC27ACYK0IX1W1FNT.png" width="70%"/>

+ Train/Val accuracy
<img  src="images/CNN/CUYI265APLFWEI1OMF6LAWC9RPGB4XDM.png" width="70%"/>

+ Ratio of weights:updates
The last quantity you might want to track is the **ratio of the update magnitudes to to the value magnitudes**. Note: updates, not the raw gradients (e.g. in vanilla sgd this would be the gradient multiplied by the learning rate). You might want to evaluate and track this ratio for every set of parameters independently. A rough heuristic is that this ratio should be somewhere around 1e-3. If it is lower than this then the learning rate might be too low. If it is higher then the learning rate is likely too high.
<img  src="images/CNN/BCKKUC9UEUMK2BLV9VTL4VW2WL67DP26.png" width="70%"/>

+ First-layer Visualizations
<img  src="images/CNN/SHUSWN24FCL9AKQJ92AA6865RBY95AF8.png" width="70%"/>

+ Data Augmentation

    1. Flip horizontally
    2. Random crops/scales
    3. Random mix/combinations of translation, rotation, stretching, shearing,lens distortions
    4. Color jittering
    


##In Practice
GPUs are very good at local, parallel operations
+ CUDA
+ higher-level API (e.g. cuBLAS, cuDNN)
+ cuda-convnet2 and caffe


<img  src="images/CNN/FUNOPP37EQWDFBI9IO4GGER7J18HLCRG.png" width="70%"/>


Data Augmentation in paper:

We employ two distinct forms of data augmentation, both of which allow transformed images to be produced from the original images with very little computation, so the transformed images do not need to be stored on disk. In our implementation, the transformed images are generated in Python code on the CPU while the GPU is training on the previous batch of images. So these data augmentation schemes are, in effect, computationally free.